In [6]:
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException, WebDriverException

def search_cbdc_articles(base_url='https://bm.ge'):
    x = input("Enter a search keyword: \n")
    search_url = f"{base_url}/news/search?phrase={x}"
    
    options = webdriver.ChromeOptions()
    options.add_argument('--start-maximized')
    options.add_argument('--disable-extensions')
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    
    max_retries = 3
    for attempt in range(max_retries):
        try:
            driver = webdriver.Chrome(options=options)
            driver.get(search_url)
            
            def click_load_more():
                try:
                    load_more_button = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, "button.load-more-btn.bg-main.w-full.h-16.text-white.font-avenir.font-medium.rounded-xl.shadow-md"))
                    )
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", load_more_button)
                    time.sleep(1)
                    load_more_button.click()
                    time.sleep(2)
                    return True
                except (TimeoutException, NoSuchElementException, ElementClickInterceptedException):
                    return False
            
            while click_load_more():
                pass
            
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            
            articles = soup.find_all('a', class_='bpg-web-caps font-medium leading-4 break-words')
            for ind, article in enumerate(articles, 1):
                title = article.text.strip().replace("\n", " ")
                print(f"############ {ind} ############")
                print("Title:", title)
                article_url = article["href"]
                article_response = requests.get(article_url)
                article_soup = BeautifulSoup(article_response.content, 'html.parser')
                desc = article_soup.find("div", class_="[&>p]:mb-4 bpg-rioni next-news-text-content news-content-for-split leading-6 overflow-hidden description")
                
                if desc:
                    print(desc.text)
                    print(type(desc.text))
                else:
                    print("Description not found")
                time.sleep(1)
            
            break  # თუ ყველაფერი წარმატებით შესრულდა, გამოვდივართ ციკლიდან
        
        except WebDriverException as e:
            print(f"An error occurred with the web driver (attempt {attempt + 1}/{max_retries}): {e}")
            if attempt < max_retries - 1:
                print("Retrying...")
                time.sleep(5)
            else:
                print("Max retries reached. Exiting.")
        finally:
            driver.quit()

search_cbdc_articles('https://bm.ge')

Enter a search keyword: 
 cbdc


############ 1 ############
Title: რას ფიქრობს FED-ის ხელმძღვანელი ინფლაციაზე, მონეტარულ პოლიტიკასა და ციფრულ დოლარზე?
პლანეტის ყველაზე მნიშვნელოვანმა ცენტრალურმა ბანკმა, ამერიკის შეერთებული შტატების ფედერალურმა რეზერვმა მონეტარულ პოლიტიკასთან დაკავშირებით გადაწყვეტილება მიიღო. ქვეყანაში რეფინანსირების განაკვეთი ისევ უცვლელად, 5.5%-იან ნიშნულზე დარჩა, რაც ანალიტიკოსების პროგნოზითაც მოსალოდნელი იყო.FED-ის გადაწყვეტილებაზე არანაკლებ საინტერესო იყო უწყების ხელმძღვანელის, ჯერომ პაუელის მოსმენა, რომელმაც ფედერალური რეზერვის სამომავლო სტრატეგიასთან დაკავშირებით ისაუბრა და საზოგადოებას თავისი პოზიცია გააცნო.„ფედერალური რეზერვის მთავარი მიზნები უცვლელია - ჩვენი ამოცანა ფასების სტაბილურობის უზრუნველყოფა და დასაქმების დონის მაქსიმალურად გაზრდაა. ბოლო პერიოდის განმავლობაში ქვეყნის ეკონომიკამ სერიოზული პროგრესი განიცადა ამ მიზნების მიღწევის მიმართულებით, მოგეხსენებათ, ინფლაციის დონე მკვეთრადაა შემცირებული, სამუშაო ადგილები კი სტაბილურად იქმნება, რაც მისასალმებელი ამბავია. მიუხედავად ყველაფრისა, ინ